In [5]:
import pandas as pd
import os
import itertools
import xlsxwriter
from rapidfuzz import fuzz

os.chdir('M:\GY number duplicates')
pd.set_option('display.max_columns', 1000, 'display.width', 1000, 'display.max_rows',1000)
demogs = df = pd.read_csv('GY_update.txt', '\t')

In [6]:
demogs = demogs.dropna(subset = ['SocialSecurityNumber']) # Ensure no NaNs in SSN column
demogs = demogs.drop_duplicates() # Ensure no duplicates

demogs = demogs[demogs['SocialSecurityNumber'].map(demogs['SocialSecurityNumber'].value_counts() > 1)] # SSN DataFrame containing duo=p SSN rows
demogs.fillna('', inplace=True)
groups = demogs.groupby('SocialSecurityNumber') # Groupby SSN

master_list = []
for group_name, df_group in groups:
    gy_group_list = df_group.astype(str).values.tolist() # Each group as a list (group) of lists (rows)
    master_list.append(gy_group_list)

cols = demogs.columns.tolist()
cols.append('Ratio')
cols.append('Diffs')

df_results = pd.DataFrame(columns = cols)
master_score = []

for i in range(len(master_list)): # For each list (group) in master_list
    number_of_rows = len(master_list[i]) # Define number of rows in group
    master_list[1]
    if number_of_rows == 2:
        score_list = []
        diffs_list = []
        for x, y in zip(master_list[i][0][1:], master_list[i][1][1:]): # For each row in group minus URN
            score = fuzz.ratio(x, y)
            score_list.append(score)

            splitB = set(x.split(' '))
            splitA = set(y.split(' '))
            diff = splitB.symmetric_difference(splitA)
            if diff != '':
                diff = ", ".join(diff) 
                diffs_list.append(diff)
            
    total_score = sum(score_list)
    master_score.append(total_score)
    
    master_list[i][0].append(int(total_score))
    master_list[i][1].append(int(total_score))
    
    
#     for feild in diffs_list:
        
#         if len(feild.split('|')) > 0 & len(diffs_list) < 2:
#             diffs_list0 = [feild.split('|')[0] for feild in diffs_list]
#             diffs_list0 = list(filter(None, diffs_list0))
#             master_list[i][0].append(diffs_list0)
#             print(diffs_list0)

#             diffs_list1 = [feild.split('|')[1] for feild in diffs_list]
#             diffs_list1 = list(filter(None, diffs_list1))
#             master_list[i][1].append(diffs_list1)
#             print(diffs_list1)
        
#     else:

    diffs_list = list(filter(None, diffs_list))

    diffs_string = "        ".join(diffs_list)
    master_list[i][0].append(diffs_string)
    master_list[i][1].append(diffs_string)

    df_results.loc[len(df_results)] = master_list[i][0]
    df_results.loc[len(df_results)] = master_list[i][1]

df_results.sort_values(by= ['Ratio', 'SocialSecurityNumber'], ascending = [False, False], inplace = True) # sort within the groups (ratio)

writer = pd.ExcelWriter("GY_Number_Dups.xlsx", engine='xlsxwriter') # Create a Pandas Excel writer using XlsxWriter as the engine

# Convert the dataframe to an XlsxWriter Excel object. Turn off the default
# header and index and skip one row to allow us to insert a user defined
# header
df_results.to_excel(writer, sheet_name='Sheet1', startrow=1, header=False, index=False)

workbook = writer.book # Get the xlsxwriter workbook and worksheet objects
worksheet = writer.sheets['Sheet1']

(max_row, max_col) = df_results.shape # Get the dimensions of the dataframe

column_settings = [] # Create a list of column headers, to use in add_table() 
for header in df_results.columns:
    column_settings.append({'header': header})

worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings,
                                                'style': 'Table Style Light 11'}) # Add the table

worksheet.set_column(0, max_col - 1, 12) # Make the columns wider for clarity.

writer.save() # Close the Pandas Excel writer and output the Excel file


df_results


In [203]:
diffs_list

['Gregory, gregory', 'Stevens-Cox, cox', ', Row, English, 3', ', Alderney']